In [1]:
import os
import sys
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

In [ ]:
csv_dir = r"E:\XD\P4V\ed_yuhang_eng\GameProject\Saved\Profiling\MemReports\UEDPIE_0_Etheria-Windows-03.13-13.40.53\Output"
bytes_format = "KB"
if bytes_format == "KB":
    bytes_divider = 1024
elif bytes_format == "MB":
    bytes_divider = 1024 * 1024
else:
    bytes_divider = 1
    
# Get a list of all CSV files in the directory
csv_map = {}

# Loop through each CSV file and plot its data
for csv_file in [f for f in os.listdir(csv_dir) if f.endswith('.csv')]:
    # Read CSV file into a pandas DataFrame
    df = pd.read_csv(f'{csv_dir}/{csv_file}')
    csv_map[csv_file] = df
    
    print(f'csv_file : {csv_file}')
    HistKeys = []
    for Col in df.columns:
        if Col.startswith('Val'):
            HistKeys.append(Col)
        
    #print(f'History Keys : {HistKeys}')
    VarNames = df['Name']
    BaseLineValues = []
    
    # Get base line values
    for i in range(len(df)):
        for hist in HistKeys[:1]:
            BaseLineValues.append(df.loc[i, hist])
            
    assert len(BaseLineValues) == len(VarNames)
            
    # Get history values, calculate Max and last value in history
    HistoryValuesForAllVar = []
    MaxValues = []
    LastValues = []
    for i in range(len(df)):
        Values = []
        for hist in HistKeys:
            Values.append(df.loc[i, hist])
        HistoryValuesForAllVar.append(Values)
        MaxValues.append(max(Values[1:])) # Calculate max value excluding the base value
        LastValues.append(Values[-1]) # Get Last value from history
        
    # Calculate Max and last value against base value
    assert len(BaseLineValues) == len(MaxValues)
    MaxBaseDiffValues = [MaxValues[i] - BaseLineValues[i] for i in range(len(BaseLineValues))]
    LastBaseDiffValues = [LastValues[i] - BaseLineValues[i] for i in range(len(BaseLineValues))]
        
    # Sort and find greatest memory influencer
    num_top = 5
    datas = [MaxBaseDiffValues] #[MaxBaseDiffValues, LastBaseDiffValues]
    for data in datas:
        np_data = np.array(data)
        sort_index = np.argsort(np_data)[::-1] # sort acending, thus reverse
        top_VarIndex = sort_index[:num_top]
        
        Top_HistoryValues = np.asarray(HistoryValuesForAllVar)[top_VarIndex]
        Top_BaseLineValues = np.asarray(BaseLineValues)[top_VarIndex]
        Top_VarNames = VarNames[top_VarIndex].to_list()
        
        print(f'Top Memory Var Names : {Top_VarNames}')
        print(f'Their diff against base (bytes) : {np_data[top_VarIndex]}')
        
        for i in range(len(Top_VarNames)):
            var_name = Top_VarNames[i]
            values = (Top_HistoryValues[i] - Top_BaseLineValues[i]) / bytes_divider
            plt.plot(range(len(values)), values, label = var_name)
            plt.xticks(range(len(values)))
            plt.xlabel("Frames")
            plt.ylabel(f"{bytes_format}")
            plt.legend()
            plt.title("Diff againt base over time")

        fig = plt.figure()
        
        
        plt.show()

    print("\n")